# Gene Annotation & Biological Insights

This notebook continues the previous analysis by mapping top-ranked probe sets to their corresponding gene symbols and descriptions.

The goal is to translate statistical outputs into meaningful biological context.

---

## Objectives

- Map Affymetrix probe IDs (e.g., `207175_at`) to human gene symbols (e.g., `FOXC1`)
- Retrieve gene titles, functional descriptions, and aliases where possible
- Create a clean summary table with:
  - `probe_id`, `gene_symbol`, `gene_title`, `log2FC`, `p_value`
- Identify genes with highest differential expression
- Interpret biological relevance of the results


In [1]:
# 📦 Import core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 📋 Set plot style
sns.set(style="whitegrid")

# ✅ Set pandas display options for better readability
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


In [12]:
# 📁 Корректные пути
diff_path = "/kaggle/input/differential-expression-results/differential_expression_results.csv"
gpl_path = "/kaggle/input/gpl570-annotation/GPL570-annotation.txt"

# 📌 Загрузка таблиц
dif_res = pd.read_csv(diff_path)
annotation_df = pd.read_csv(gpl_path, sep="\t", comment="#", low_memory=False)

# 👁️ Предпросмотр обеих таблиц
print("🔬 Differential Expression Results:")
display(dif_res.head())

print("\n📚 Annotation Table Preview:")
display(annotation_df[["ID", "Gene Symbol", "Gene Title"]].head())


🔬 Differential Expression Results:


,probe_id,mean_normal,mean_cancer,log2FC,p_value,abs_log2FC
0,207175_at,11.130027,4.736829,-1.232464,2.503821e-05,1.232464
1,217428_s_at,4.002313,8.994421,1.168196,3.442948e-39,1.168196
2,1552509_a_at,7.801721,3.485118,-1.162585,4.803564e-04,1.162585
3,209613_s_at,10.089049,4.598064,-1.133692,9.273542e-05,1.133692
4,209773_s_at,4.106258,8.945309,1.123307,2.579246e-10,1.123307



📚 Annotation Table Preview:


,ID,Gene Symbol,Gene Title
0,1007_s_at,DDR1 /// MIR4640,discoidin domain receptor tyrosine kinase 1 //...
1,1053_at,RFC2,"replication factor C (activator 1) 2, 40kDa"
2,117_at,HSPA6,heat shock 70kDa protein 6 (HSP70B')
3,121_at,PAX8,paired box 8
4,1255_g_at,GUCA1A,guanylate cyclase activator 1A (retina)


In [13]:
# ✅ Merge differential results with annotation data
merged_df = pd.merge(
    dif_res,
    annotation_df[["ID", "Gene Symbol", "Gene Title"]],
    left_on="probe_id",
    right_on="ID",
    how="left"
)

final_annotated_df = merged_df[[
    "probe_id", "Gene Symbol", "Gene Title", "log2FC", "p_value"
]].copy()

# 👁️ Preview
print("🧬 Annotated Differential Expression Table:")
display(final_annotated_df.head())


🧬 Annotated Differential Expression Table:


,probe_id,Gene Symbol,Gene Title,log2FC,p_value
0,207175_at,ADIPOQ,"adiponectin, C1Q and collagen domain containing",-1.232464,2.503821e-05
1,217428_s_at,COL10A1,"collagen, type X, alpha 1",1.168196,3.442948e-39
2,1552509_a_at,CD300LG,CD300 molecule-like family member g,-1.162585,4.803564e-04
3,209613_s_at,ADH1B,"alcohol dehydrogenase 1B (class I), beta polyp...",-1.133692,9.273542e-05
4,209773_s_at,RRM2,ribonucleotide reductase M2,1.123307,2.579246e-10


In [14]:
# 🔍 Count how many times each gene symbol appears (how many probes map to each gene)
gene_counts = final_annotated_df["Gene Symbol"].value_counts().reset_index()
gene_counts.columns = ["gene_symbol", "count"]

# 🔁 Filter to keep only genes covered by more than one probe
repeated_genes = gene_counts[gene_counts["count"] > 1]

# 📌 Display results
print("📌 Genes covered by multiple probes:")
display(repeated_genes)

# 💾 Optionally export the list
repeated_genes.to_csv("/kaggle/working/repeated_gene_symbols.csv", index=False)


📌 Genes covered by multiple probes:


,gene_symbol,count
0,HFE,15
1,TCF3,13
2,LOC100506403 /// LOC101928269 /// RUNX1,13
3,CD44,13
4,CSNK1A1,13
...,...,...
11266,MYB,2
11267,ZBTB1,2
11268,DAPK1,2
11269,BAALC,2


In [16]:
# 👁️ Pick a real gene that exists in repeated_genes
repeated_genes.head(10)


,gene_symbol,count
0,HFE,15
1,TCF3,13
2,LOC100506403 /// LOC101928269 /// RUNX1,13
3,CD44,13
4,CSNK1A1,13
5,DNAH1,13
6,ARHGEF12,13
7,MALAT1,12
8,GNAS,12
9,MAGI1,12


In [17]:
final_annotated_df.to_csv("/kaggle/working/annotated_differential_results.csv", index=False)
